- Assume we've already made the summaries
- So we only pass in the recent file changes, and ask it to autocomplete
- The response schema is the same though


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from groq import Groq
import json
import os
import watchdog
import asyncio
from src.loader import get_dir_summaries

In [3]:
os.environ["GROQ_API_KEY"] = "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"

In [40]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEvent, FileSystemEventHandler


class Watcher:
    def __init__(self, base_path, callback):
        self.observer = Observer()
        self.base_path = base_path
        self.callback = callback

    def run(self):
        event_handler = Handler(self.base_path, self.callback)
        self.observer.schedule(event_handler, self.base_path, recursive=True)
        self.observer.start()
        try:
            while True:
                time.sleep(5)
        except KeyboardInterrupt:
            self.observer.stop()
            print(f"Observer on directory {self.base_path} stopped")
        self.observer.join()


class Handler(FileSystemEventHandler):
    def __init__(self, base_path, callback):
        self.base_path = base_path
        self.callback = callback
        self.events = []

        print(f"Watching directory {base_path}")

        # Hack to get async function to work
        loop = asyncio.new_event_loop()
        # Ensure the async setup is run thread-safely
        future = loop.run_until_complete(get_doc_summaries(base_path))
        self.summaries = future.result()  # This will block until the coroutine is done
        print(self.summaries)

    def on_created(self, event: FileSystemEvent) -> None:
        src_path = os.path.relpath(event.src_path, self.base_path)
        print(f"Created {src_path}")
        # self.callback(event.src_path)

    def on_deleted(self, event: FileSystemEvent) -> None:
        src_path = os.path.relpath(event.src_path, self.base_path)
        print(f"Deleted {src_path}")
        # self.callback(event.src_path)

    def on_moved(self, event: FileSystemEvent) -> None:
        src_path = os.path.relpath(event.src_path, self.base_path)
        dest_path = os.path.relpath(event.dest_path, self.base_path)
        print(f"Moved {src_path} > {dest_path}")
        self.events.append({"src_path": src_path, "dst_path": dest_path})
        self.callback(self.summaries, {"files": self.events})

    # def on_any_event(self, event):
    #     print(event)
    #     print(event.event_type, event.src_path, event.dest_path)
    #     if not event.is_directory:
    #         self.callback(event.src_path)

In [41]:
# BASE_PATH = "./sample_data"
# summaries = await get_doc_summaries(BASE_PATH)

In [42]:
FILE_PROMPT = """
You will be provided with list of source files and a summary of their contents. For each file, propose a new path and filename, using a directory structure that optimally organizes the files using known conventions and best practices.

If the file is already named well or matches a known convention, set the destination path to the same as the source path.

Your response must be a JSON object with the following schema:
```json
{
    "files": [
        {
            "src_path": "original file path",
            "dst_path": "new file path under proposed directory structure with proposed file name"
        }
    ]
}
```
""".strip()


WATCH_PROMPT = """
Here are a few examples of good file naming conventions to emulate, based on the files provided:

```json
{fs_events}
```

Include the above items in your response exactly as is, along all other proposed changes.
""".strip()

In [43]:
def create_file_tree(summaries, fs_events):
    client = Groq()
    cmpl = client.chat.completions.create(
        messages=[
            {"content": FILE_PROMPT, "role": "system"},
            {"content": json.dumps(summaries), "role": "user"},
            {"content": WATCH_PROMPT.format(fs_events), "role": "system"},
            {"content": json.dumps(summaries), "role": "user"},
        ],
        model="llama3-70b-8192",
        response_format={"type": "json_object"},
        temperature=0,
    )
    print(cmpl.choices[0].message.content)
    return json.loads(cmpl.choices[0].message.content)

In [44]:
w = Watcher("./sample_data", create_file_tree)
w.run()

Watching directory ./sample_data


/Users/iyaja/mambaforge/lib/python3.9/ast.py:50: RuntimeWarning: coroutine 'get_doc_summaries' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: Cannot run the event loop while another loop is running